In [ ]:
import sys
import json
import os
import chardet

In [ ]:
from typing import Dict
from pydantic import BaseModel, ValidationError
from itertools import islice

In [ ]:
class UserData(BaseModel):
    user_id: str
    category: str

In [ ]:
def detect_file_encoding(file_path: str) -> str:
    """Detect encoding."""
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read())
    return result['encoding']

In [ ]:
def read_file_to_dict(file_path: str) -> Dict[str, str]:
    """
    Read txt, convert to dict, print two rows.

    Task:
        Задание 1
        Переведите содержимое файла purchase_log.txt
        в словарь purchases вида:
        {‘1840e0b9d4’: ‘Продукты’, …}

        Пример работы программы при выводе первых
        двух элементов словаря purchases:
        1840e0b9d4 ‘Продукты‘
        4e4f90fcfb  ‘Электроника‘


    Args:
        file_path: path to the file.

    Returns:
        A dictionary with keys and values from the file.
    """
    output_dict = {}

    encoding = detect_file_encoding(file_path)

    with open(file_path, "r", encoding=encoding) as file:
        # skip header
        next(file)

        for line_num, line in enumerate(file, 2):
            try:
                data = UserData(**json.loads(line.strip()))
                output_dict[data.user_id] = data.category
            except ValidationError as e:
                print(f"Error on line {line_num}: {e}")
            except json.JSONDecodeError:
                print(f"Error decoding JSON on line {line_num}.")

    return output_dict

In [ ]:
def first_two_items(d: Dict) -> Dict:
    """get two rows."""
    return dict(islice(d.items(), 2))

In [ ]:
def run_homework():
    is_jupyter = 'ipykernel' in sys.modules
    if is_jupyter:
        DEFAULT_FILE_PATH = os.path.join(os.getcwd(), 'purchase_log.txt')
    else:
        DEFAULT_FILE_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'purchase_log.txt')
    try:
        purchases = read_file_to_dict(DEFAULT_FILE_PATH)
        if not purchases:
            print("The file does not contain valid data entries after the header.")
        else:
            output_purchases = first_two_items(purchases)
            for key, value in output_purchases.items():
                print(f"{key} '{value}'")
    except FileNotFoundError:
        print(f"File '{DEFAULT_FILE_PATH}' not found.")

In [ ]:
if __name__ == "__main__":
    run_homework()